In [1]:
import os
import boto3
import json

In [2]:
bucket_name = 'woi-images'
bucket_region = 'eu-west-1'
SNSTopicArn = 'arn:aws:sns:region:************:video_resp'
RoleArn = 'arn:aws:iam::************:role/user/user_name'

In [3]:
rek_client = boto3.client('rekognition', bucket_region)
s3_client = boto3.resource('s3', bucket_region)
s3_bucket = s3_client.Bucket(bucket_name)
file_list = [obj.key for obj in s3_bucket.objects.all()]

In [17]:
job_file_name = 'Jobs.txt'
if not os.path.exists(job_file_name):
    open(job_file_name, 'x').close()
index = 0    

Amazon Rekognition has a limit for API request. We can make 20 start_person_tracking requests simultaneously.

In [137]:
open(job_file_name, 'w').close()

for file in file_list[index:min(index+20,len(file_list))]:
    start_ind = file.index('/')
    end_ind = [i for i,val in enumerate(file) if val=='_'][2]
    response = rek_client.start_person_tracking(
    Video={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': file
        }
    },
    ClientRequestToken=file[start_ind+1:end_ind],
    NotificationChannel={
        'SNSTopicArn': SNSTopicArn,
        'RoleArn': RoleArn
    },
    JobTag=file[start_ind+1:end_ind]
)
    if response["ResponseMetadata"]["HTTPStatusCode"] == 200:
        with open(job_file_name, "a") as myfile:
            myfile.write(response["JobId"]+"\n")
index += 20            

In [138]:
index

300

In [142]:
lines = open('Jobs.txt').readlines()

last_resp =  rek_client.get_person_tracking(
    JobId=lines[len(lines)-1].rstrip()
)

last_resp["JobStatus"]

'SUCCEEDED'

In [143]:
if last_resp["JobStatus"] == 'SUCCEEDED':
    for i in range(len(lines)):
        resp = rek_client.get_person_tracking(
        JobId=lines[i].rstrip()
    )
        if resp["JobStatus"] == 'SUCCEEDED':
            res = "./"+file_list[index-20+i][:-4]+'.json'
            os.makedirs(os.path.dirname(res), exist_ok=True)
            with open(res, 'w') as outfile:
                json.dump(resp, outfile)